In [1]:
from cProfile import label

import numpy as np
import matplotlib.pyplot as plt
import h5py
from dask.array import unique
from networkx.classes import non_neighbors

from main import plot_neurons_in_light, plot_light_level

%matplotlib qt
%load_ext autoreload
%autoreload 2

## Loading data and saving it as npy file

In [ ]:
neuron_data_file = "/media/smb/soma-fs.ad01.caesar.de/bbo/projects/SugiWallace-lightdark/tuning_curves.mat"
neuron_data = h5py.File(neuron_data_file, 'r')
neuorn_data_dict = {
    'neuron_activity': np.asarray(neuron_data['darkcells_kin'], dtype=np.float32).T,
    'lightlevel_bins': np.asarray(neuron_data['edg']).flatten(),
    'lightlevel': np.asarray(neuron_data[neuron_data['lightmask'][5,0]]).flatten()
}

np.save("neuron_data.npy", neuorn_data_dict, allow_pickle=True)

## Analysis

In [64]:
neuorn_data_dict =  np.load("neuron_data.npy", allow_pickle=True).item()

light_level = np.log10(neuorn_data_dict['lightlevel'])
neuron_activity = neuorn_data_dict['neuron_activity']
n_samples, n_features = neuron_activity.T.shape
light_period = 2160 # roughly
#neuron_activity[neuron_activity == 0] = np.nan

light_level_discrete = np.digitize(light_level, bins=neuorn_data_dict['lightlevel_bins'])

In [3]:
plot_light_level(light_level, neuorn_data_dict['lightlevel_bins'])

In [4]:
plot_neurons_in_light(neuron_activity, light_level)

In [5]:
import scipy.stats as stats

freq, bins = np.histogram(light_level, bins=neuorn_data_dict['lightlevel_bins'])
ll_entropy = stats.entropy(freq / np.sum(freq), base=2)
ll_entropy

np.float64(1.8382505277114545)

In [18]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

In [36]:
X_train, X_test, y_train, y_test  = train_test_split(neuron_activity.T, light_level_discrete, test_size=0.33, random_state=3, shuffle=False)

In [39]:
# Use a fixed random_state for reproducibility
model = LogisticRegression(random_state=3, max_iter=5000, penalty='l2')
#unique, counts = np.unique(y_train, return_counts=True)
#freq_map = dict(zip(unique, 1 / counts))
#arr_weights = np.vectorize(freq_map.get)(y_train)
model.fit(X_train, y_train) #, sample_weight=arr_weights)
y_predict = model.predict(X_test)

print("Hold-out accuracy:", accuracy_score(y_test, y_predict))

cv_scores = cross_val_score(
    model,
    neuron_activity.T,
    light_level_discrete,
    cv=3,
    n_jobs=10,
    scoring='accuracy',
)
print("CV fold accuracies:", cv_scores)
print("CV mean accuracy:", cv_scores.mean())


Hold-out accuracy: 0.47264597580220935


/home/cheekoti/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/cheekoti/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/prepr

CV fold accuracies: [0.68463542 0.55598958 0.39166667]
CV mean accuracy: 0.5440972222222222


/home/cheekoti/miniforge3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
cm = confusion_matrix(y_test, y_predict, normalize='true')
ConfusionMatrixDisplay(cm).plot()

In [16]:
param_grid_dict = {
    #'max_iter': np.linspace(10, 100, 10, dtype=int),
    #'max_depth': np.linspace(3, 10, 7, dtype=int),
    #'criterion': ['gini', 'entropy', 'log_loss'],

}

grid = GridSearchCV(RandomForestClassifier(), param_grid_dict, cv=3, n_jobs=15, scoring='accuracy')
grid.fit(neuron_activity.T, light_level_discrete)

,estimator,RandomForestClassifier()
,param_grid,"{'criterion': ['gini', 'entropy', ...], 'max_depth': array([ 3, 4..., 7, 8, 10]), 'n_estimators': array([ 10, ...80, 90, 100])}"
,scoring,'accuracy'
,n_jobs,15
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,np.int64(20)


In [27]:
plt.figure()
plt.plot(grid.cv_results_['mean_test_score'])
plt.plot(grid.cv_results_['param_n_estimators']/2000 + 0.65)

In [43]:
from sklearn.decomposition import PCA, KernelPCA

pca = PCA(random_state=3)
X_pca = pca.fit_transform(neuron_activity.T)
plt.figure()
plt.plot(pca.explained_variance_ratio_, '.-', label='Individual')
plt.plot(np.cumsum(pca.explained_variance_ratio_), '.-', label='Cumulative')
plt.xlabel('Component index')
plt.ylabel('Explained variance ratio')
plt.legend()


In [41]:
print("Explained variance:", pca.explained_variance_ratio_)
print("Cumulative:", np.cumsum(pca.explained_variance_ratio_))

Explained variance: [0.36043292 0.08157275 0.0548402 ]
Cumulative: [0.36043292 0.44200566 0.49684587]


In [74]:
for i in range(n_samples // light_period):
    plt.figure()
    ax = plt.axes(projection='3d')

    ax.scatter(X_pca[i*light_period:(i+1)*light_period, 0], X_pca[i*light_period:(i+1)*light_period, 1], X_pca[i*light_period:(i+1)*light_period, 2], c=light_level_discrete[i*light_period:(i+1)*light_period].astype(float)/np.max(light_level_discrete))
    #ax.scatter(X_pca[:x_size, 0], X_pca[:x_size, 1], X_pca[:x_size, 2], c=light_level_discrete[:x_size].astype(float)/np.max(light_level_discrete))
    plt.title(f"Period {i}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    #plt.zlabel("PC3")
    plt.show()

In [75]:
plt.figure()
ax = plt.axes(projection='3d')

ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=light_level_discrete.astype(float)/np.max(light_level_discrete))
plt.title(f"All")
plt.xlabel("PC1")
plt.ylabel("PC2")
#plt.zlabel("PC3")
plt.show()

## Information theory

In [ ]:
from sklearn import feature_selection

mi_regression = feature_selection.mutual_info_regression(neuron_activity.T, light_level, n_neighbors=5, n_jobs=10)

In [ ]:
mi_classif = feature_selection.mutual_info_classif(neuron_activity.T, light_level_discrete,  n_neighbors=5, n_jobs=10)

In [ ]:
plt.figure()
plt.plot(mi_regression, label='Mutual information for regression')
plt.plot(mi_classif, label='Mutual information for classification')
plt.xlabel('Neuron index')
plt.ylabel('Mutual information')

plt.axhline(y=ll_entropy, color='r', linestyle='-', label='Entropy of light level')
plt.legend(loc='upper right')

In [ ]:
from pyitlib import discrete_random_variable as drv

drv.entropy_joint(X=neuron_activity)

In [ ]:
import infomeasure as im
print(im.entropy(
    tuple(neuron_activity), approach='kernel', kernel='gaussian', bandwidth=1.0, base=2))
print(im.entropy(light_level_discrete, approach='kernel', kernel='gaussian', bandwidth=1.0, base=2))

In [ ]:
im.mutual_information(*neuron_activity[:10], base=2, approach='kernel')#, embedding_dim=3)

In [ ]:
from infomeasure.estimators import mutual_information

ord_mi_est = mutual_information.OrdinalMIEstimator(*neuron_activity, embedding_dim=10, base=2)
ord_mi_est_vars = vars(ord_mi_est)


In [ ]:
im.mutual_information(*neuron_activity, base=2, approach='metric', k=2), im.entropy(
    tuple(neuron_activity), approach='metric', k = 2, base=2)

In [ ]:
im.mutual_information(*neuron_activity, base=2, approach='metric', k=5), im.entropy(
    tuple(neuron_activity), approach='metric', k = 5, base=2)